## load DII plain text of patents

In [1]:
import os,glob

In [ ]:
def file_load_f(input_path):
    '''input_path'''
    file_path = []
    for dir_name, dirs, files in os.walk(input_path):
        if len(files) > 0:
            for input_file in (glob.glob(dir_name + '\*.txt')):
                file_path.append(input_file)
    return file_path

## extract patent ab infor

In [2]:
def ab_collection_f(file):
    ab_data =[]
    ab_temp = []
    for line in file.readlines():
        if line[0:2] == 'AB':
            is_in = 1
            start_line = ([line[6:-1]])
            ab_temp.append(start_line)
        elif line[0:3] == '   ' and is_in == 1:
            follow_line = line[3:-1]
            ab_temp.append(start_line.append(follow_line))
        else:
            is_in = 0
    for x in ab_temp:
        if x != None  :
            ab_data.append(x)
    return ab_data

## extract patent ti infor

In [4]:
def ti_collection_f(file_path):
    file = open(file_path, encoding='utf-8')
    ti_collection=[]
    for line in file.readlines():
        if line.startswith('TI '):
            ti_collection.append(str(line[3:-1]))
    return ti_collection

## divide the ab infor to "NOVELTY" "USE" "ADVANTAGE"

In [6]:
def ab_mess_process_f(ab_data_slice):
    ad_extract=[]
    i = 0
    if len(ab_data_slice)<3:
        mark = 3 - len(ab_data_slice)
        for x in ab_data_slice:
            if x.strip().startswith('(') and 'NOVELTY' in x.strip():
                ad_extract.append(x.strip().split(')')[1].strip())
                ad_extract.append('miss')
                ad_extract.append('miss')
                continue
            if x.strip().startswith('NOVELTY'):
                ad_extract.append(x)
            elif x.strip().startswith('USE'):
                ad_extract.append(x)
            elif x.strip().startswith('ADVANTAGE'):
                ad_extract.append(x)
            else:
                ad_extract.append('miss')
        for i_add in range(mark):
            ad_extract.append('miss')
    else:
        for x in ab_data_slice:
            if x.strip().startswith('(') and 'NOVELTY' in x.strip():
                ad_extract.append(x.strip().split(')')[1].strip())
                ad_extract.append('miss')
                ad_extract.append('miss')
                continue
            if x.strip().startswith('NOVELTY'):
                ad_extract.append(x)
            elif x.strip().startswith('USE'):
                ad_extract.append(x)
            elif x.strip().startswith('ADVANTAGE'):
                ad_extract.append(x)
            else:
                ad_extract.append('miss')
            i += 1
            if i == 3:
                break
    return ad_extract

## massive ab infor process

In [8]:
def ab_process_f(ab_data):
    ad_normmalize = []
    for x in ab_data:
        #print(len(x))
        ad_normmalize.append(ab_mess_process_f(x))
    return ad_normmalize

## create CSV file

In [12]:
def createListCSV_ab(fileName, header=None, data_List=[]):
    with open(fileName, 'w', newline='') as csvFile:
        import csv
        csvWriter = csv.writer(csvFile)
        if header:
            csvWriter.writerow(header)
        for data in data_List:
            for y in data:
                csvWriter.writerow([y[0].split('- ')[-1],y[1].split('- ')[-1],y[2].split('- ')[-1]])
        csvFile.close

In [ ]:
def createListCSV_ti(fileName, header=None, data_List=[]):
    with open(fileName, 'w', newline='') as csvFile:
        import csv
        csvWriter = csv.writer(csvFile)
        if header:
            csvWriter.writerow(header)
        for data in data_List:
            if  not isinstance(data,list):
                csvWriter.writerow([data])
        csvFile.close

# application

In [ ]:
input_path = r''#input_path--where your file is stored such as "C:\Users\bandk\Desktop\graphene_dii\2012 " a folder path
otput_path = r''#output_path--where your file will be output in CSV such as "C:\Users\bandk\Desktop\graphene_dii\2012_out_ab " a folder path
file_path  =file_load_f(input_path)
    ti_data =[]
    for f in file_path:
        file = open(f, 'r', encoding='UTF-8')
        ab_data_temp = ab_process_f(ab_collection_f(file))
        ab_data.append(ab_data_temp)
        ti_data_temp = ti_collection_f(file)
        ti_data = ti_data+ti_data_temp
    header = ['NOVELTY','USE','ADVANTAGE']
    createListCSV_ab(os.path.join(output_path,'ab_extract.csv'),header,ab_data) 
    header = ['TI']
    createListCSV_ti(os.path.join(output_path,'ti_extract.csv'),header,ab_data) 